# Differentiation case study

    Thomas Moreau <thomas.moreau@inria.fr>
    Alexandre Gramfort <alexandre.gramfort@inria.fr>

We will showcase the 2 modes of automatic differentiation on a simple function

$
\begin{cases}
    f_0(x) = x\\
    f_{k+1}(x) = 4 f_k(x) ( 1 - f_k(x))
\end{cases}
$

Here is a simple implementation:

In [1]:
def f(x, n=4):
    v = x
    for _ in range(n):
        v = 4 * v * (1 - v)
    return v

f(0.25)

0.75

## Numerical differentiation

In [2]:
from scipy.optimize import approx_fprime
approx_fprime(0.25, f, 1e-7)

array([-16.00001599])

## Forward differentiation

The idea is to use this formula:

$$
\frac{\partial f_{k+1}}{\partial x}(x)
= \frac{\partial f_{k+1}}{\partial f_{k}}(f_{k}) \frac{\partial f_{k}}{\partial x} (x)
= \frac{\partial f_{k+1}}{\partial f_{k}}(f_{k})
    \left[
    \frac{\partial f_{k}}{\partial f_{k-1}}(f_{k-1})
    \dots
    \frac{\partial f_{1}}{\partial x}(x)
    \right]
$$

In [3]:
def g(x, n=4):
    v, dv = x, 1.
    for i in range(n):
        dv = (4  - 8 * v) * dv
        v = 4 * v * (1 - v)
    return dv

g(0.25)

-16.0

## Backward differentiation

The idea is to use this formula:

$$
\frac{\partial f_{k+1}}{\partial x}(x)
= \frac{\partial f_{k+1}}{\partial f_{1}}(f_1) \frac{\partial f_{1}}{\partial x} (x)
= 
    \left[
    \frac{\partial f_{k+1}}{\partial f_{k}}(f_{k})
    \dots
    \frac{\partial f_{k}}{\partial f_{1}}(f_{1})
    \right]
    \frac{\partial f_{1}}{\partial x}(x)
$$

In [4]:
def g(x, n=4):
    v = x
    memory = []
    for i in range(n):
        memory.append(v)
        v = 4 * v * (1 - v)
    dv = 1
    for v in memory[::-1]:
        dv = dv * (4  - 8 * v)
    return dv

g(0.25)

-16.0

<div class="alert alert-success">

**Exercice:**

- Code the forward and backward propagation for the following function defined for $x\in]0, 1[$:

$
\qquad
\begin{cases}
f_0(x) = x\\
f_{n+1}(x) = 1 + \frac{2}{f_{n}(x)}
\end{cases}
$
    
_For any $x > 0$, the limit of this series gives $2$ so the derivative will be 0 for $n$ large enough_
</div>

**Note:** the derivative of $g(x) = 1 + \frac{2}{x}$ is $g'(x) = \frac{-2}{x^2}$.

In [5]:
# Definition of the function.
def f(x, n=4):
    v = x
    for _ in range(n):
        v = 1 + 2 / v

    return v

x = 2
f(x)

2.0

Solution in `solutions/01a_forward_diff.py`

In [7]:
# Gradient of the function f computed with forward diff
def g_forward(x, n=4):
    v, dv = x, 1
    for _ in range(n):
        dv =  -2 * dv / v ** 2
        v = 1 + 2 / v

    return dv

x = 2
g_forward(x), approx_fprime(x, f, 1e-9)[0]


(0.0625, 0.06250022204458655)

In [8]:
# Check solution forward diff
import numpy as np

diff = np.array([
    abs(g_forward(x) - approx_fprime(x, f, 1e-9)[0])
    for x in np.logspace(-4, 0, 21, base=2)
])
mask = diff < 1e-6
assert mask.all(), diff[~mask]
print("Good job for the forward diff!")

Good job for the forward diff!


Solution in `solutions/01a_backward_diff.py`.

In [ ]:
def g_backward(x, n=4):
    ##############
    # TODO

    # END TODO
    ############
    return dv

x = 2
g_backward(x), approx_fprime(x, f, 1e-9)[0]

In [ ]:
# Check solution forward diff
import numpy as np

diff = np.array([
    abs(g_backward(x) - approx_fprime(x, f, 1e-9)[0])
    for x in np.logspace(-4, 0, 21, base=2)
])
mask = diff < 1e-6
assert mask.all(), diff[~mask]
print("Good job for the backward diff!")

### Bonus - network with residual structure

_This question is a bit more complex as there is a residual structure._

<div class="alert alert-success">

**Exercice:**

- Code the forward and backward propagation for the following function defined for $x\in]0, 1[$:

$
\qquad
\begin{cases}
f_0(x) = 1\\
f_{n+1}(x) = (1- x) * f_{n}(x) + 1
\end{cases}
$
    
_This corresponds to the power series of $\frac{1}{x}$ so one can check that the gradient should converge to $\frac{-1}{x^2}$._
</div>

_**Hint:** For the backward diff, the residual structure requires an accumulation! Compute separatly $\frac{\partial f_{n}}{\partial f_k}$ and $\Big(\frac{d f_{n}}{d x}\Big)_k = \sum_{i=k}^n\frac{\partial f_{n}}{\partial f_k}\frac{\partial f_k(x)}{\partial x}$. This second term corresponds to the gradient of $f_n(x)$ if $f_{k-1}(x)$ is considered as a constant relative to $x$._

In [ ]:
# Definition of the function.
def f(x, n=1000):
    v = 1
    for _ in range(n):
        v = (1 - x) * v + 1

    return v

x = .5
f(x), 1/x

Solution in `solutions/01a_bonus_forward_diff.py`

In [ ]:
# Gradient of the function f computed with forward diff
def g_forward(x, n=1000):
    ##############
    # TODO

    # END TODO
    ############
    return dv


x = .5
g_forward(x), -1 / x**2


In [ ]:
# Check solution forward diff
import numpy as np

diff = np.array([
    abs(g_forward(x) - (-1/x**2))
    for x in np.logspace(-4, 0, 21, base=2)
])
mask = diff >= 1e-10
assert not mask.any(), diff[mask]
print("Good job for the forward diff!")

Solution in `solutions/01a_bonus_backward_diff.py`.

In [ ]:
def g_backward(x, n=1000):
    ##############
    # TODO

    # END TODO
    ############
    return dv

x = .5
g_backward(x), -1 / x**2

In [ ]:
# Check solution backward diff
diff = np.array([
    abs(g_backward(x) -(-1/x**2))
    for x in np.logspace(-4, 0, 21, base=2)
])
mask = diff >= 1e-10
assert not mask.any(), diff[mask]
print("Good job for the backward diff!")